In [39]:
!pip install geopy
from tqdm import tqdm
import geopy
import pandas as pd

In [36]:
property_files = '../data/'

In [37]:
df = pd.read_csv(f'{property_files}curated/past_listing.csv')
df = df.rename(columns={'code': 'postcode'}, errors="coerce")
df = df[df.groupby('address').address.transform('count') > 1]

In [38]:
# read in the postcode information
postcode_df = pd.read_csv('../data/raw/abs/australian_postcodes.csv')
# only retain information that is relavent for faster running time
postcode_df = postcode_df.loc[postcode_df['state'] == 'VIC']
postcode_df = postcode_df[['postcode', 'lgaregion']]
df = pd.merge(df, postcode_df, on='postcode')

In [32]:
metro_melb = ['Banyule', 'Bayside', 'Boroondara', 'Brimbank', 'Cardinia', 'Casey', 'Darebin', 'Frankston', 'Glen Eira',
              'Greater Dandenong', 'Hobsons Bay', 'Hume', 'Kingston', 'Knox', 'Manningham', 'Maribyrnong', 'Maroondah',
              'Melbourne', 'Melton', 'Monash', 'Moonee Valley', 'Moreland', 'Mornington Peninsula', 'Nillumbik',
              'Port Phillip', 'Stonnington', 'Whitehorse', 'Whittlesea', 'Wyndham', 'Yarra', 'Yarra Ranges']
df = df.loc[df['lgaregion'].isin(metro_melb)]

In [33]:
len(df)

7931986

In [34]:
addresses = df.loc[df['year'] == 2022]['address'].tolist()
addresses = df.loc[df['year'] == 2021]['address'].tolist() + addresses
addresses = list(set(addresses))
len(addresses)

197044

In [10]:
# geocode_df = pd.read_csv('geocode.csv');

In [11]:
addresses[0]

'29/240 MOUNTAIN HIGHWAY WAY, WANTIRNA'

In [12]:
# geocode_df
geocode_df = []
coded_l = []
# coded_l = geocode_df.address.to_list()

In [13]:
df2 = pd.read_csv(f'{property_files}raw/geo.csv').iloc[: , 1:]
df2.head()

,address,loc_address,lat,lon
0,"5 MECKLENBURG CLOSE, EPPING","Mecklenburg Close, Lyndarum Estate, Epping, Me...",-37.636569,145.019908
1,"29/202 WATTLETREE ROAD, MALVERN","Wattletree Road, Malvern, Melbourne, City of S...",-37.863492,145.037014
2,"6/51 DAVIS AVENUE, SOUTH YARRA","Davis Avenue, South Yarra, Melbourne, City of ...",-37.840817,144.990352
3,"22 STEWART STREET, SEYMOUR","Stewart Street, Seymour, Shire of Mitchell, Vi...",-37.030939,145.140848
4,"1/10 ORCHARD ROAD, BAYSWATER","Orchard Road, Bayswater, Melbourne, City of Kn...",-37.839234,145.259483


In [14]:
len(df2)

200506

In [15]:
temp = []
addresses_saved = df2['address'].tolist()
for i in tqdm(addresses):
    if i not in addresses_saved:
        temp.append(i)

100%|██████████| 197044/197044 [04:37<00:00, 710.42it/s]


In [16]:
addresses = temp
len(addresses)

27615

In [12]:
count = 0;
geo_data = []
for address in tqdm(addresses):
    locator = geopy.Nominatim(user_agent="myGeocoder");
    location = locator.geocode(address+", VICTORIA",timeout=None);
    if location != None:
        info = [address, location.address, location.latitude, location.longitude]
        geo_data.append(info)
        df2.loc[len(df2)] = info

100%|██████████| 62353/62353 [15:59:26<00:00,  1.08it/s]   


In [17]:
df2.head()

,address,loc_address,lat,lon
0,"5 MECKLENBURG CLOSE, EPPING","Mecklenburg Close, Lyndarum Estate, Epping, Me...",-37.636569,145.019908
1,"29/202 WATTLETREE ROAD, MALVERN","Wattletree Road, Malvern, Melbourne, City of S...",-37.863492,145.037014
2,"6/51 DAVIS AVENUE, SOUTH YARRA","Davis Avenue, South Yarra, Melbourne, City of ...",-37.840817,144.990352
3,"22 STEWART STREET, SEYMOUR","Stewart Street, Seymour, Shire of Mitchell, Vi...",-37.030939,145.140848
4,"1/10 ORCHARD ROAD, BAYSWATER","Orchard Road, Bayswater, Melbourne, City of Kn...",-37.839234,145.259483


In [26]:
df2.to_csv(f'{property_files}/raw/geo.csv')